In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import nibabel as nib
import numpy as np
import os
import pathlib

In [ ]:
import os,random

In [ ]:
# !unzip "/content/drive/MyDrive/brats15"

In [ ]:
!pip install batchgenerators

In [ ]:
!pip install nnunetv2

In [ ]:
!pip install --upgrade git+https://github.com/FabianIsensee/hiddenlayer.git

In [ ]:
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_preprocessed"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/DatasetGlioblastoma"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/DatasetGlioblastoma/imagesTr"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/DatasetGlioblastoma/imagesTs"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/DatasetGlioblastoma/labelsTr"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_results"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_results/inference"
!mkdir -p "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_results/postprocessed"

In [ ]:
segm_src_dir = '/content/drive/MyDrive/TCIA/UPENN-GBM/brats15/'
segm_file_list = os.listdir(segm_src_dir)

random.shuffle(segm_file_list)

train_ratio = 0.8
elements = len(segm_file_list)
train_elements = int(elements * train_ratio)

train_segm_list = segm_file_list[:train_elements]
test_segm_list = segm_file_list[train_elements:]

print (train_segm_list)
len(train_segm_list)

In [ ]:
tr_segm_target_dir = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/labelsTr/'

for file in train_segm_list:
  tr_segm_file = file.split('_')[0]
  patient_id = tr_segm_file.split('-')[2]
  just_id = int(patient_id.split('.')[0])
  print ('!mv ' + segm_src_dir + file + '/'+ file + '-seg.nii.gz' +' ' + tr_segm_target_dir + str(just_id) + '.nii.gz')


In [ ]:
ts_segm_target_dir = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/labelsTs/'

for file in test_segm_list:
  ts_segm_file = file.split('_')[0]
  patient_id = ts_segm_file.split('-')[2]
  just_id = int(patient_id)
  print ('!mv ' + segm_src_dir + file + '/'+ file + '-seg.nii.gz' +' ' + tr_segm_target_dir + str(just_id) + '.nii.gz')

In [ ]:
! git clone https://github.com/MIC-DKFZ/nnUNet/tree/master/nnunetv2

In [ ]:
mri_img_src_dir = '/content/drive/MyDrive/TCIA/UPENN-GBM/brats15/'
tr_img_target_dir = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/imagesTr/'

for patient in train_segm_list:
  patient_id = patient.split('.')[0]
  patient_dir = patient_id
  patient_dir_path = mri_img_src_dir + patient_dir
  just_id = int(patient_id.split('-')[2])

  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t1c.nii.gz ' + tr_img_target_dir + str(just_id) + '_0000.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t1n.nii.gz ' + tr_img_target_dir + str(just_id) + '_0001.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t2w.nii.gz ' + tr_img_target_dir + str(just_id) + '_0002.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t2f.nii.gz ' + tr_img_target_dir + str(just_id) + '_0003.nii.gz')

In [ ]:
dmri_img_src_dir = '/content/drive/MyDrive/TCIA/UPENN-GBM/brats15/'
ts_img_target_dir = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/imagesTs/'

for patient in test_segm_list:
  patient_id = patient.split('.')[0]
  patient_dir = patient_id
  patient_dir_path = mri_img_src_dir + patient_dir
  just_id = int(patient_id.split('-')[2])

  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t1c.nii.gz ' + tr_img_target_dir + str(just_id) + '_0000.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t1n.nii.gz ' + tr_img_target_dir + str(just_id) + '_0001.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t2w.nii.gz ' + tr_img_target_dir + str(just_id) + '_0002.nii.gz')
  print ('!mv ' + patient_dir_path + '/' + patient_dir + '-t2f.nii.gz ' + tr_img_target_dir + str(just_id) + '_0003.nii.gz')

In [ ]:
base_img_path = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/labelsTr/'
tr_label_files = os.listdir(base_img_path)
tr_label_files.sort()
print(len(tr_label_files))
print(tr_label_files)

In [ ]:
import matplotlib.pyplot as plt

for file in tr_label_files:

  file_name = base_img_path + file
  img = nib.load(file_name)
  print("-"*100)
  print ("Patient label file: ", file)

  img_data = img.get_fdata()

  print("Before label check")
  print(np.amin(img_data),np.amax(img_data))
  print(np.unique(img_data))

plt.imshow(img_data[:,:,100], cmap='gray')
plt.colorbar()
plt.show()


In [ ]:
for file in tr_label_files:

  file_name = base_img_path + file
  img = nib.load(file_name)
  print("-"*100)
  print ("Patient label file: ", file)

  img_data = img.get_fdata()

  print("Before label check")
  print(np.amin(img_data),np.amax(img_data))
  print(np.unique(img_data))

  img_data[(img_data == 4.0)] = 3

  print("After label conversion")
  print(np.amin(img_data),np.amax(img_data))
  print(np.unique(img_data))

  new_img = nib.Nifti1Image(img_data, img.affine, img.header)

  new_img_path = base_img_path + file
  print ('Saved to: ' + new_img_path)
  nib.save(new_img, new_img_path)

In [ ]:
output_folder = '/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma'
channel_names = {0: 'T1', 1: 'T1GD', 2:'T2', 3: 'FLAIR'}
labels = {'background': 0, 'ET': 1, 'NCR': 2, 'ED': 3}
file_ending = '.nii.gz'
region_class_order = {
  'background': 0,
  'whole tumor': (1, 2, 3),
  'tumor core': (2, 3),
  'enhancing tumor': 3
}
num_training_cases = 11
dataset_name = 'UPENN-GBM'
description = 'UPENN-GBM tumor segmentation'

In [ ]:
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json
generate_dataset_json(output_folder=output_folder,channel_names=channel_names, labels=labels,
                      file_ending=file_ending,region_class_order=region_class_order,
                      num_training_cases=num_training_cases, dataset_name=dataset_name, description=description)

In [ ]:
os.environ['nnUNet_raw'] = "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw"
os.environ['nnUNet_preprocessed'] =  "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/TCIA/nnUNet/nnUNet_results"

!nnUNetv2_plan_and_preprocess -d 501 --verify_dataset_integrity

In [ ]:
! nnUNetv2_train Dataset501_Glioblastoma 3d_fullres 0 --npz

In [ ]:
!nnUNetv2_find_best_configuration DatasetGlioma -c 3d_fullres

In [ ]:
!nnUNetv2_predict -d Dataset501_Glioblastoma -i /content/drive/MyDrive/TCIA/nnUNet/nnUNet_raw/Dataset501_Glioblastoma/imagesTs -o /content/drive/MyDrive/TCIA/nnUNet/nnUNet_results/Dataset501_Glioblastoma/inference -f  0 1 2 3 4 -tr nnUNetTrainer -c 3d_fullres -p nnUNetPlans